In [3]:
import numpy as np
import cmdstanpy
import astropy.units as u
import h5py

In [4]:
from astropy.io import fits

In [12]:
# rcat = 'YOUR DATA HERE! Make sure that the necessary columns (used below) are present, or modify the code below as appropriate.'
rcat = fits.open('/home/js/programs/gmestan/data/h3/h3_nogalprior_v42.fits')[1].data

In [11]:
rcat[rcat['FLAG'] == 0].size

168

In [17]:
rca = np.array(rcat)

In [22]:
import pandas as pd

In [24]:
df = pd.DataFrame(rca)

In [25]:
df.to_csv('../data/h3.csv')

In [9]:
filtered = rcat[np.logical_and.reduce((
    rcat['FLAG'] == 0,  
    rcat['SNR'] > 3, 
    rcat['Vrot'] < 5,
    rcat['Teff'] < 7000,
    rcat['V_tan'] < 1000,
    np.abs(rcat['V_gsr']) < 400,
    rcat['R_gal'] > 50,

    ~np.isnan(rcat['GAIAEDR3_RA']), 
    ~np.isnan(rcat['GAIAEDR3_PMRA']), 
    ~np.isnan(rcat['GAIAEDR3_PARALLAX']),
    ~np.isnan(rcat['Vrad']),
    ~np.isnan(rcat['X_gal']),
))]

filtered.size

168

In [ ]:
posobs = (~np.isnan(filtered['GAIAEDR3_RA'])).astype('int')
pmobs = (~np.isnan(filtered['GAIAEDR3_PMRA'])).astype('int') 
distobs = (~np.isnan(filtered['dist_adpt'])).astype('int')
vlosobs = (~np.isnan(filtered['Vrad'])).astype('int') 
d = np.nan_to_num(filtered, 0)

In [ ]:
# MULTIPLY kpc by 1000 to get pc
# DIVIDE mas by 1000 to get arcsec

standata = {
    
    'N': len(d),
    'pos_obs': posobs,
    'dist_obs': distobs,
    'pm_obs': pmobs,
    'vlos_obs': vlosobs,

    'ra_measured': d['GAIAEDR3_RA'], 
    'dec_measured': d['GAIAEDR3_DEC'],
    'dist_measured': d['dist_adpt'], # kpc
    
    'ra_err': d['GAIAEDR3_RA_ERROR'] * u.mas.to(u.deg), 
    'dec_err': d['GAIAEDR3_DEC_ERROR'] * u.mas.to(u.deg),
    'dist_err': d['dist_adpt_err'], # kpc

    'Xgc': d['X_gal'],
    'Ygc': d['Y_gal'],
    'Zgc': d['Z_gal'],

    'pmra_measured': d['GAIAEDR3_PMRA'], # mas/yr
    'pmdec_measured': d['GAIAEDR3_PMDEC'], # mas/yr
    'vlos_measured': d['Vrad'],

    'pmra_err': d['GAIAEDR3_PMRA_ERROR'], # mas/yr
    'pmdec_err': d['GAIAEDR3_PMDEC_ERROR'], # mas/yr
    'vlos_err': d['Vrad_err'],
    
    'pos_corr': d['GAIAEDR3_RA_DEC_CORR'],
    'pm_corr': d['GAIAEDR3_PMRA_PMDEC_CORR'],
    'ra_pmra_corr': d['GAIAEDR3_RA_PMRA_CORR'],
    'dec_pmdec_corr': d['GAIAEDR3_DEC_PMDEC_CORR'],
    'ra_pmdec_corr': d['GAIAEDR3_RA_PMDEC_CORR'],
    'dec_pmra_corr': d['GAIAEDR3_DEC_PMRA_CORR'],

    'grainsize': 1,
    
    # try different priors!
    'alpha_mean': 4.,
    'alpha_sigma': 0.1,
    'beta_mean': 0.3,
    'beta_sigma': 0.05,
    
}

In [ ]:
sm = cmdstanpy.CmdStanModel(
    stan_file='../models/h3p.stan', 
    stanc_options={'include_paths': ['../models']},
    cpp_options={"STAN_THREADS": False, "STAN_OPENCL": True},
)
sm

In [ ]:
def init_gen():
    return {

        'p_gamma': 0.4,
        'p_phi0': 65,
        'p_beta': 0.5,
        'p_alpha': 4.,

        'pmra': standata['pmra_measured'] / 5,
        'pmdec': standata['pmdec_measured'] / 5,
        'dist': standata['dist_measured'],
        'vlos': standata['vlos_measured'] / 5,
    }

In [ ]:
fit = sm.sample(
    data=standata, 
#     iter_warmup=1,
#     iter_sampling=1,
    chains=2, 
#     adapt_delta=0.,
#     adapt_init_phase=1000,
#     adapt_metric_window=1000,
#     adapt_step_size=1000,
#     threads_per_chain=4, 
    inits=init_gen(),
    show_progress='notebook',
    save_profile=True,
    output_dir='../saved',
#     max_treedepth=11,
)

### Refer to `gme_py.ipynb` for some examples of what to do next!